# Title

# Overview

Overview here

# Group Members

 - Ivan Jin (A14880280)
 - Hongsheng Xie (A14794772)
 - Tong Wang (A13713688)
 - Yinxuan Du (A15873678)
 - Yuchen Zhang (A16151373)

# Research Question

Research Question Here

# Background & Prior Work

BG + Prior work here

# Hypothesis

Hypothesis here

# Datasets

### 1. Airline Delay and Cancellation Data, 2009 - 2018
- Content: US domestic flight delay data
- Columns of interest: 
- \# of observations: 6429338 rows for year 2009
- Source: https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018
- Function: main outcome variable

### 2. FILL ME ACCORDING TO OUR DATASET!
- Content:
- Columns of interest: 
- \# of observations: 
- Source: 
- Function:

# Setup

In [3]:
import pandas as pd
import numpy as np
from numpy import nan as Nan
import matplotlib.pyplot as plt
import requests
from string import digits
import wget 
import glob
import time

import plotly.graph_objects as go
import pandas as pd
import plotly.figure_factory as ff
import xml.etree.ElementTree as ET # to read one dataset in XML format

# Data Cleaning

### Overview

To answer our question, we need to clean and aggregate our main outcome variable (delay, crash) and our predictors by airports. The goal of data cleaning is to get two dataframes that have airport code as index:
 - 1.Y 
     - columns
         - average_departure_delay - average delay of all flights originated from this airport 
         - average_arrival_delay - average delay of all flights arriving at this airport
         - accident_count - number of accidents in which aircraft originated from this airport
 - 2.X 
     - columns
         - code4 - 4 digit airport code
         - city_name - name of the city that located near the airport
         - city_id - index of this city in uscities_df
         - latitude - latitude of the airport
         - longitude - longitude of the airport
         - altitude_ft - altitude of the airport
         - temp_avg - 2018-2019 average tempreature of the county contining the airport
         - pcp_avg - 2018-2019 average precipitation of the county contining the airport
         - strike_avg - year-average number bird strike (may not caused damage)
         - damage_avg - year-average number of bird strike that caused damage
         - city_population - population the city that located near the airport
         - enplanements - enplanements of the city that located near the airport
         - length_ft_sum - sum of length of all runways
         - width_ft_avg - average width of all runways
         - num_runways - number of runways

### Cleaning - Y

#### Delay dataset
From `A Pleasant Flight.ipynb` (now in the folder `previous_codes`)

In [55]:
delay_2018_df = pd.read_csv("datasets/original/delay/2018.csv")
delay_2018_df = delay_2018_df[["ORIGIN","DEST","DEP_DELAY","TAXI_OUT","CANCELLED","DISTANCE","ARR_DELAY","TAXI_IN"]] # columns of interest

First,we check the distribution of missing values:

In [56]:
delay_2018_df.isna().sum()

ORIGIN            0
DEST              0
DEP_DELAY    117234
TAXI_OUT     115830
CANCELLED         0
DISTANCE          0
ARR_DELAY    137040
TAXI_IN      119246
dtype: int64

There are missing values for delay columns. However, we discovered that 95% of the missing value have "CANCALLED" == 1. It is logical to have missing delay value when the flight is never done. For these flight have already contributed to the cancel rate feature, thus should not have a effect on the delay. Since we are using the .mean() function, which will ignore all nan values, we left these rows unchanged. Same for diverted, when there shouldn't be a arrival delay because the flight is not arriving at the scheduled airport. We kept them because we can have a more accurate total departure/arrival number. In this way we have only 4000 rows that we can't explain the reason behind missing value. Since we have 7213446 rows in total, 4000 is a acceptable amount of missing. We also kept it for accurate total departure/arrival number.
We then calculated departure_delay, arrival_delay and other columns of interest. 

In [61]:
# DEPARTURE
unpleasant_2018_departure = pd.DataFrame()
unpleasant_2018_departure['total_departure'] = delay_2018_df.loc[:,["ORIGIN"]].groupby('ORIGIN').size()
unpleasant_2018_departure[["average_departure_delay","average_departure_taxi"]] = delay_2018_df.loc[:,["ORIGIN","DEP_DELAY","TAXI_OUT"]].groupby('ORIGIN').mean()
unpleasant_2018_departure['average_departure_cancelled'] = delay_2018_df.loc[:,["ORIGIN","CANCELLED"]].groupby('ORIGIN').mean()
unpleasant_2018_departure['averge_departure_distance'] = delay_2018_df.loc[:,["ORIGIN","DISTANCE"]].groupby('ORIGIN').mean()
    
#ARRIVAL
unpleasant_2018_arrival = pd.DataFrame()
unpleasant_2018_arrival['total_arrival'] = delay_2018_df.loc[:,["DEST"]].groupby('DEST').size()
unpleasant_2018_arrival[["average_arrival_delay","average_arrival_taxi"]] = delay_2018_df.loc[:,["DEST","ARR_DELAY","TAXI_IN"]].groupby('DEST').mean()
unpleasant_2018_arrival['average_arrival_diverted'] = delay_2018_df.loc[:,["DEST","DIVERTED"]].groupby('DEST').mean()
unpleasant_2018_arrival['averge_arrival_distance'] = delay_2018_df.loc[:,["DEST","DISTANCE"]].groupby('DEST').mean()

#Add log to total_departure and total_arrival 
unpleasant_2018_departure['total_departure_lg10'] = unpleasant_2018_departure['total_departure'].apply(np.log10)
unpleasant_2018_arrival['total_arrival_lg10'] = unpleasant_2018_arrival['total_arrival'].apply(np.log10)

#Merge departure and arrival
unpleasant_2018 = unpleasant_2018_departure.merge(unpleasant_2018_arrival,left_index=True,right_index=True)
unpleasant_2018.index.names = ["airport_code"]

#### Air Crash dataset
From `A Pleasant Flight.ipynb` (now in the folder `previous_codes`)

In [62]:
xml_tree = ET.parse("datasets/original/crash/AviationData.xml") # NTSB crash dataset


# Data Analysis & Results

## EDA

## Analysis

# Ethics & Privacy

Ethics here

# Conclusion

Conclusion here